## Startup
Replace with path to `neuropixel-utils` repo

In [5]:
addpath(genpath('~/code/neuropixel-utils'))

Set the default channel map file, this one is located in `neuropixel-utils/map_files`

In [8]:
setenv('NEUROPIXEL_MAP_FILE', which('neuropixPhase3A_kilosortChanMap.mat')); 

This part is optional. You can arrange your data however you like. Below we'll use `Neuropixel.generatePath` to generate paths automatically below this root folder:

In [12]:
setenv('NEUROPIXEL_DATAROOT', '~/data/neuropixel');

# Load Raw Imec dataset

Here we construct the path to the original, raw dataset as it was recorded by SpikeGLX. You don't need to use this folder nesting structure.

In [17]:
subject = 'TestSubject';
dateStr = '2018-12-02';
rawBinFile = 'TestSubject_20181202_01.imec.ap.bin';
imecFile = Neuropixel.generatePath(subject, 'raw_datasets', dateStr, rawBinFile)


imecFile =

    '~/data/neuropixel/TestSubject/raw_datasets/2018-12-02/TestSubject_20181202_01.imec.ap.bin'



We then use this path to construct an ImecDataset object which will facilitate access to the raw data and load the metadata from disk. You can pass the path to the imec dataset in several ways:
* path to raw `.imec.ap.bin` file or `imec.lf.bin` file
* path to the containing directory, if there is only one set of `.bin` files within

In [19]:
imec = Neuropixel.ImecDataset(imecFile)


imec = 

  ImecDataset with properties:

              pathRoot: '~/data/neuropixel/TestSubject/raw_datasets/2018-12-02'
              fileStem: 'TestSubject_20181202_01'
          creationTime: 7.3722e+05
             nChannels: 385
            fileTypeAP: 'ap'
            nSamplesAP: 112412208
            nSamplesLF: 0
                  fsAP: 30000
                  fsLF: NaN
      highPassFilterHz: 300
                apGain: 500
               apRange: [-0.6000 0.6000]
                lfGain: 250
               lfRange: [-0.6000 0.6000]
               adcBits: 10
            channelMap: [1x1 Neuropixel.ChannelMap]
      syncChannelIndex: 385
          syncInAPFile: 1
           badChannels: []
          syncBitNames: {16x1 cell}
               syncRaw: []
        bytesPerSample: 2
                 hasAP: 1
                 hasLF: 0
        channelMapFile: '/Users/djoshea/npl/neuropixel-utils/map_files/neuropixPhase3A_kilosortChanMap.mat'
        mappedChannels: [384x1 double]
    

## Specifying the channel map manually

By default, the channel map will be loaded from whatever the environment variable `NEUROPIXEL_MAP_FILE` is set to, which we set above. If you want to manaully specify a channel map, pass it in as:

```matlab
imec = Neuropixel.ImecDataset(imecFile, 'channelMap', '/path/to/channelMap.mat');
```

## Explore the loaded metadata

Here we see that this option 3A probe has 384 total channels (saved in the `.imec.ap.bin` file), and that 374 of these channels are connected, namely those listed in `.connectedChannels`.

In [20]:
imec.channelMap


ans = 

  ChannelMap with properties:

                 file: '/Users/djoshea/npl/neuropixel-utils/map_files/neuropixPhase3A_kilosortChanMap.mat'
              chanMap: [384x1 double]
          chanMap0ind: [384x1 double]
            connected: [384x1 logical]
             shankInd: [384x1 double]
              xcoords: [384x1 double]
              ycoords: [384x1 double]
              zcoords: [384x1 double]
            nChannels: 384
    connectedChannels: [374x1 double]
    referenceChannels: [10x1 double]
             yspacing: 20
             xspacing: 16
             zspacing: [0x1 double]



We can determine the duration of the recording as:

In [26]:
fprintf('Duration of recording %s is %g minutes\n', imec.fileStem, imec.nSamplesAP / imec.fsAP / 60)

Duration of recording TestSubject_20181202_01 is 62.4512 minutes


# Accessing the raw AP data

You can access the raw data in several ways (see the full list of methods using `methods(imec)`). The most straightforward is to memory map the full binary data:

In [ ]:
NeuropixelExpt.DataLoad.setImecSyncBitNames(imec);

if ~isempty(p.Results.badChannels)
    imec.markBadChannels(p.Results.badChannels);
end

rmsBadChannels = imec.markBadChannelsByRMS('rmsRange', [3 100]);
debug('Marking %d channels bad based on rms\n', numel(rmsBadChannels));


% save the bad channels and sync bit names to the meta file
imec.writeModifiedAPMeta();

% CAR the file and flush the unused sync bits to the cleaned_datasets
% folder
cleanedPath = NeuropixelExpt.generatePath(subject, 'cleaned_datasets', dateStr, p.Results.cleanedBinFile);

debug('Writing CAR version at %s\n', cleanedPath);
extraMeta = struct();
extraMeta.run_clearUnusedSyncBits = true;
extraMeta.run_detectAndMarkStimArtifactWindows = true;
fnList = {@NeuropixelExpt.DataClean.clearUnusedSyncBits; @Neuropixel.DataProcessFn.commonAverageReference};
imec = imec.saveTransformedDataset(cleanedPath, ...
    'goodChannelsOnly', false, 'writeSyncSeparate', false, ...
    'transformAP', fnList, 'extraMeta', extraMeta);

% sym link into ks directory
ksPath = NeuropixelExpt.generatePath(subject, 'ks', dateStr, p.Results.cleanedBinFile);
debug('Sym-linking to %s\n', ksPath);
imec = imec.symLinkAPIntoDirectory(ksPath);

% and run Kilosort
debug('Running Kilosort\n');
Neuropixel.runKilosort2(imec);

info.rawPath = imecFile;
info.cleanedPath = cleanedPath;
info.ksPath = ksPath;

end